In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [29]:
## Load dataset

data = pd.read_csv('Churn_Modelling.csv')

In [30]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
## Preprocessing, dropping irrelevant columns

data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [32]:
## Encoding categorical data
labelencoder = LabelEncoder()
data['Gender'] = labelencoder.fit_transform(data['Gender'])


In [33]:
## One hot encoding

data = pd.get_dummies(data, columns=['Geography'])

In [34]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


In [35]:
## Saving label encoder and one hot encoder

with open('labelencoder.pkl', 'wb') as file:
    pickle.dump(labelencoder, file)

with open('onehotencoder.pkl', 'wb') as file:
    pickle.dump(data, file)

In [37]:
## Splitting dataset into training and testing

X= data.drop('Exited', axis=1)

y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Feature scaling

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [38]:
## Saving the scaler

with open('scaler.pkl', 'wb') as file:
    pickle.dump(sc, file)

## Importing tensorflow and training model

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential  # importing Sequential model
from tensorflow.keras.layers import Dense  # importing Dense layers for the model
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard  # importing EarlyStopping for the model
from tensorflow.keras.layers import Dropout  # importing Dropout layers for the model
import datetime  # importing datetime for the model 

In [46]:
## building the ANN model
# creating a Sequential model   
model = Sequential([
   Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## Hidden layer 1 connected to input layer
   Dense(32, activation='relu'), ## Hidden layer 2
   Dense(1, activation='sigmoid') ## Output layer
])    



In [49]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 64)                832       
_________________________________________________________________
dense_10 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 33        
Total params: 2,945
Trainable params: 2,945
Non-trainable params: 0
_________________________________________________________________


In [52]:
## Defining the optimizer and loss function

opt = tf.keras.optimizers.Adam(learning_rate=0.01)

loss = tf.keras.losses.BinaryCrossentropy()

In [53]:
## Compiling the model

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])